# Predicting Change in Insulin Consumption Among Medicare Beneficiaries

## EconS 524 Applied Machine Learning for Economics

### Pari Magphanthong

## 1. Background

In 2020, over 3.26 million Medicare beneficiaries depend on one or more forms of insulin to manage their medical conditions. The aggregated out-of-pocket cost for insulin alone was \\$1.03 billion. A Medicare beneficiary spends \\$572 out of pocket to cover the cost of insulin annually . Insulin is a life-saving medication and a critical component to medical management for many people. Cost burden increases the gap in insulin access which poses a significant risk on medical complications from unmanaged diabetes, such as amputation or heart attack . Center for Medicare and Medicaid Services (CMS) recognized the severe consequences from insulin inaccessibility. CMS implemented the Part D Senior Savings Model (SSM) on January 1, 2021. SSM offers supplementary insulin coverage which caps the out-of-pocket expense for insulin at \$35 per one-month supply during the deductible, initial coverage, and coverage gap phase. SSM aims to close the insulin access gap and improve medication adherence.

## 2. Objective

The objective of this project is to utilize multiple machine learning algorithms to predict the change in average 30-day insulin consumption among Medicare beneficiaries in response to the SSM.

## 3. Data

The final dataset I use for analysis consists of 10 years of microdata, from 2010-2019, about insulin consumption and expenditure, and comorbid conditions. There are 9,004 observations in this dataset, each observation is a sampled individual. It is important to note that this is a cross-sectional dataset where each observation in different survey year is not the same individual. The samples included in this analysis are limited to those who filled at least one insulin prescription during a given survey year.

The final dataset is comprised of three separate datasets: demand and expenditure, prescription details, and comorbid conditions. All of the dataset are from the Medical Expenditure Panel Survey (MEPS). The following section display examples of each dataset from the year 2010. The datasets are the same format across all the included survey years. However, due to the size of the datasets and the limited computational capability, I only include examples from 2010 to better understand the structure of the data.

### 3.1 Demand and Expenditure

This dataset contains data about insulin demand, out of pocket expenditure, and insurance coverages. This is a hierrachical data where each row of observation is of different unit type, data from one individual span across many rows. Consider the example below, all the row displayed contain data about a sampled individual in survey year 2010, let us called him A:

* Row 0 has a *RECTYPE* P meaning that it contains overall demographic and socioeconomic information of A, including his survey ID, age, age at first diagnosed with diabetes, household income, race, insurance coverage (beneficiaries of Medicare, Medicaid, private insurance, etc.), and other per individual information.
* Row 1 has a *RECTYPE* M meaning that it contains specific information about A's household. The main purpose of this type of data is for survey tracking, not applicable to this analysis.
* Row 2-4 has a *RECTYPE* F meaning that each row contains information per one prescription filled, including what type of medicine was filled, how much was paid by A, how much was paid by each of his insurance, how many day does that prescription fill supplied, etc.

The main purpose of this analysis is to predict demand per individual per 30-day. As such, I aggregated all type F data for any individual to create an aggregate insulin demand data per individual per survey year.

In [2]:
import numpy as np
import pandas as pd

# Example
diabetes10 = pd.read_excel('/Users/parimagphanthong/Library/CloudStorage/OneDrive-WashingtonStateUniversity(email.wsu.edu)/EconS 701 Capstone/diabetes2010.xlsx')

pd.set_option('display.max_columns', None) # set to display all columns

diabetes10.head()

,RECTYPE,SERIAL,PERNUM,YEAR,DUID,PID,MEPSID,PANEL,PSUANN,STRATANN,PSUPLD,STRATAPLD,PANELYR,RELYR,PERWEIGHT,SAQWEIGHT,DIABWEIGHT,AGE,SEX,RACEA,HISPETH,FTOTVAL,FOODSTYN,HINOTCOV,HIPRIVATE,HICHAMPANY,HIMACHIP,HIMCARE,COVERTYPE,DIABETICAGE,INSULIN,YEARM,SERIALM,DUIDM,PIDM,PANELM,DUPERSIDM,MEPSIDM,LINKIDM,MEPSLINKIDM,NREFILLS,YEARF,SERIALF,DUIDF,PIDF,PANELF,DUPERSIDF,MEPSIDF,DRUGID,MEPSDRUGID,RXRECID,MEPSRXRECID,LINKIDF,MEPSLINKIDF,RXNAME,RXDAYSUP,RXFEXPSRC,RXFEXPSELF,RXFEXPMC,RXFEXPMA,RXFEXPPRI,RXFEXPVA,RXFEXPTRI,RXFEXPOF,RXFEXPOL,RXFEXPWC,RXFEXPOS,MULTC1S1,MULTC1S1S1
0,P,1,1,2010.0,10007.0,101.0,1.510007e+09,15.0,2.0,20101024.0,2.0,1024.0,2010.0,1.0,4859.254882,5134.367675,0.0,29.0,1.0,100.0,30.0,67000.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,96.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,1.0,10007.0,101.0,15.0,10007101.0,1.510007e+09,1.000710e+11,1.510007e+15,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,F,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,1.0,10007.0,101.0,15.0,10007101.0,1.510007e+09,1.000710e+10,1.510007e+12,1.000710e+14,1.510007e+18,1.000710e+11,1.510007e+15,ATENOLOL,9999.0,1.0,4.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,274.0
3,F,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,1.0,10007.0,101.0,15.0,10007101.0,1.510007e+09,1.000710e+10,1.510007e+12,1.000710e+14,1.510007e+18,1.000710e+11,1.510007e+15,ATENOLOL,9999.0,1.0,4.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,274.0
4,F,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2010.0,1.0,10007.0,101.0,15.0,10007101.0,1.510007e+09,1.000710e+10,1.510007e+12,1.000710e+14,1.510007e+18,1.000710e+11,1.510007e+15,ATENOLOL,9999.0,1.0,4.83,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47.0,274.0


### 3.2 Prescription details

This dataset contains the breakdown of each insulin prescription filled including strength and quantity. Each row is a prescription filled so insulin prescription data for an individual in a survey year spans across many rows. I, again, aggregated prescription quantity for any individual to create an aggregate insulin quantity supplied data per individual per survey year.

In [3]:
# Example
insulin10 = pd.read_excel('/Users/parimagphanthong/Library/CloudStorage/OneDrive-WashingtonStateUniversity(email.wsu.edu)/EconS 701 Capstone/insulin2010.xlsx')


insulin10.head()

,DUID,PID,DUPERSID,DRUGIDX,RXRECIDX,LINKIDX,PANEL,PURCHRD,RXBEGDD,RXBEGMM,RXBEGYRX,RXNAME,RXNDC,RXQUANTY,RXFORM,RXFRMUNT,RXSTRENG,RXSTRUNT,RXDAYSUP,PHARTP1,PHARTP2,PHARTP3,PHARTP4,PHARTP5,PHARTP6,PHARTP7,RXFLG,PCIMPFLG,CLMOMFLG,INPCFLG
0,10007,101,10007101,10007101001,100071010031001,100071010031,15,1,-1,-1,2005,ATENOLOL,68382002210,30.0,TABS,TAB,25,MG,-8,4,-1,-1,-1,-1,-1,-1,1,2,1,0
1,10007,101,10007101,10007101001,100071010031002,100071010031,15,1,-1,-1,2005,ATENOLOL,68382002210,30.0,TABS,TAB,25,MG,-8,4,-1,-1,-1,-1,-1,-1,1,2,1,0
2,10007,101,10007101,10007101001,100071010031003,100071010031,15,1,-1,-1,2005,ATENOLOL,68382002210,30.0,TABS,TAB,25,MG,-8,4,-1,-1,-1,-1,-1,-1,1,2,1,0
3,10007,101,10007101,10007101002,100071010041001,100071010041,15,1,25,1,2010,AZITHROMYCIN,59762314001,30.0,SUSR,ML,200,MG/5ML,-8,4,-1,-1,-1,-1,-1,-1,1,2,1,0
4,10007,101,10007101,10007101003,100071010081001,100071010081,15,3,20,12,2010,ATENOLOL,51079068463,90.0,TAB,OTHER,50,MG,90,2,4,-1,-1,-1,-1,-1,1,2,2,0


### 3.3 Cormobid conditions

This dataset contains the cormorbid conditions and other health related conditions. Each row is an individual.

In [4]:
# Example
comorbid10 = pd.read_excel('/Users/parimagphanthong/Library/CloudStorage/OneDrive-WashingtonStateUniversity(email.wsu.edu)/EconS 701 Capstone/comorbid2010.xlsx')


comorbid10.head()

,DUID,PID,DUPERSID,CONDN,CONDIDX,PANEL,CONDRN,AGEDIAG,REMISSN,CRND1,CRND2,CRND3,CRND4,CRND5,INJURY,ACCDENTD,ACCDENTM,ACCDENTY,ACCDNJAN,ACCDNWRK,MISSWORK,MISSSCHL,INBEDFLG,ICD9CODX,ICD9PROX,CCCODEX,HHNUM,IPNUM,OPNUM,OBNUM
0,10007,101,10007101,11,100071010011,15,1,24,-1,1,0,1,-1,-1,2,-1,-1,-1,-1,-1,0,0,0,401,-1,98,0,0,0,0
1,10007,101,10007101,21,100071010021,15,1,27,-1,1,0,0,-1,-1,2,-1,-1,-1,-1,-1,0,0,0,272,-1,53,0,0,0,0
2,10007,101,10007101,41,100071010041,15,1,-1,-1,1,0,0,-1,-1,2,-1,-1,-1,-1,-1,0,0,0,473,-1,126,0,0,0,1
3,10007,101,10007101,51,100071010051,15,1,-1,-1,1,0,0,-1,-1,2,-1,-1,-1,-1,-1,0,0,0,794,-1,151,0,0,0,0
4,10007,102,10007102,21,100071020021,15,1,-1,-1,1,0,0,-1,-1,2,-1,-1,-1,-1,-1,0,0,0,346,-1,84,0,0,0,0


### 3.4 Aggregated data

After aggregating and cleaning all three data sets for each survey year, I combined all of them by matching each individual by their unique survey ID. The final data set contains out of pocket expenditure, coverage by primary insurance type (dollar amount) and quantity consumed per 30-day supply. It also contains other demographic and socioeconomic status.

In [6]:
# Upload finalized cleaned data
insulin = pd.read_excel('/Users/parimagphanthong/Library/CloudStorage/OneDrive-WashingtonStateUniversity(email.wsu.edu)/EconS 701 Capstone/final_insulin.xlsx')

# Overview of data
pd.set_option('display.max_columns', None) # set to display all columns
insulin.head()

,DUPERSID,self_ex,medicare_x,medicaid_x,private_x,avgdayquant,otherhi,year,RXRECID,age,race,hispanic,hhincome,diabetes_age,daysupply,RXBEGYRX,RXQUANTY,RXFRMUNT,RXSTRENG,RXSTRUNT,heart_disease,obesity,hypertension,lipidem,no_insurance_dummy,private_y_dummy,other_dummy,medicaid_y_dummy,medicare_y_dummy,female,nonhisp_white,nonhisp_black,hisp,other_race,lself_ex,Unnamed: 0,ex_hhincome,catastrophic,sumquant,sumdays,PERWEIGHT,priminsurance,noinsurance,primmedicare,primmedicaid,primprivate,primother,month,monthcost,over35,under35,avgmonthquant,strength,total_price,self_pct,medicare_pct,medicaid_pct,private_pct,other_pct,primpay
0,10053102,0.0,0.0,564.42,0.00,0.356537,0.0,2010,100531020141001,47,100,30,0.9704,30,30,2004,15.0,ML,100,UNIT/ML,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0.000000,NaN,NaN,NaN,210.0,589,2606.397216,medicaid_x,0,0,1,0,0,19.633333,0.000000,0,1,10.696095,0,564.42,0.000000,0.0,100.000000,0.000000,0.0,28.748048
1,10053102,0.0,0.0,564.42,0.00,0.658996,0.0,2010,100531020141001,47,100,30,0.9704,30,30,2004,15.0,ML,100,UNIT/ML,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0.000000,NaN,NaN,NaN,315.0,478,2895.143310,medicaid_x,0,0,1,0,0,15.933333,0.000000,0,1,19.769874,0,564.42,0.000000,0.0,100.000000,0.000000,0.0,35.423849
2,10066102,7.0,0.0,0.00,134.48,1.000000,0.0,2010,100661020051001,73,100,10,4.7700,50,30,2007,30.0,ML,100,UNIT/ML,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1.945910,NaN,NaN,NaN,30.0,30,6514.246582,private_x,0,0,0,1,0,1.000000,7.000000,0,1,30.000000,0,141.48,4.947696,0.0,0.000000,95.052304,0.0,134.480000
3,10066102,7.0,0.0,0.00,134.48,0.333333,0.0,2010,100661020051001,73,100,10,4.7700,50,30,2007,30.0,ML,100,UNIT/ML,0,0,0,0,0,1,0,0,1,0,1,0,0,0,1.945910,NaN,NaN,NaN,30.0,90,6860.221191,private_x,0,0,0,1,0,3.000000,2.333333,0,1,10.000000,0,141.48,4.947696,0.0,0.000000,95.052304,0.0,44.826667
4,10073101,4.5,0.0,998.91,0.00,0.679245,0.0,2010,100731010131001,30,200,50,0.9062,18,25,1999,20.0,ML,100,UNIT/ML,0,0,0,0,0,1,0,1,0,1,0,0,1,0,1.504077,NaN,NaN,NaN,180.0,265,2502.778808,medicaid_x,0,0,1,0,0,8.833333,0.509434,0,1,20.377358,0,1003.41,0.448471,0.0,99.551529,0.000000,0.0,113.084151


### 3.5 Descriptive statistics

The summary statistics below shows the demographics, socioeconomic, and consumption data broken down by type of primary insurance and whether or not an individual pays over $35 for a 30-day supply.

It also shows that the mean 30-day consumption is higher among those who pay more than $35 and those who pay less than that.

In [7]:
# SUMMARY STATISTICS
stat = pd.DataFrame()
categories_sum = ['female','nonhisp_white', 'nonhisp_black','hisp', 'other_race']
categories_med = ['age', 'hhincome', 'diabetes_age', 'self_ex', 'monthcost', 'avgmonthquant', 'primpay']
categories_len = ['obs']
insurances = ['no_insurance_dummy','primmedicare', 'primmedicaid','primprivate', 'primother', 'over35', 'under35']

# ['female','nonhisp_white', 'nonhisp_black','hisp', 'other_race']
for category in categories_sum:
    for insurance in insurances:
        stat.loc[category,'overall'] = insulin[category].sum()
        stat.loc[category, insurance] = insulin.loc[insulin[insurance] == 1, category].sum()
        
# ['med_age', 'med_hhincome', 'diabetes_age', 'oop','qinsulin']
for category in categories_med:
    for insurance in insurances:
        stat.loc[category,'overall'] = insulin[category].median()
        stat.loc[category, insurance] = insulin.loc[insulin[insurance] == 1, category].median()
# ['obs']
for category in categories_len:
    for insurance in insurances:
        stat.loc[category,'overall'] = len(insulin)
        stat.loc[category, insurance] = len(insulin.loc[insulin[insurance] == 1])
        
print(stat)
        
# Mean difference
print('\n Mean difference in quantity consumed per 30-day between those who pay more than $35 and less than $35')
mean_diff = insulin.groupby('over35')['avgmonthquant'].mean()
print(mean_diff)

# Make proportion
## Summing all amount paid
total_price = ['self_ex', 'medicare_x','medicaid_x','private_x','otherhi']
insulin['total_price'] = insulin[total_price].sum(axis=1)
insulin = insulin[insulin['total_price']>0]
## Make proportion
insulin['self_pct'] = (insulin['self_ex']/insulin['total_price'])*100
insulin['medicare_pct'] = (insulin['medicare_x']/insulin['total_price'])*100
insulin['medicaid_pct'] = (insulin['medicaid_x']/insulin['total_price'])*100
insulin['private_pct'] = (insulin['private_x']/insulin['total_price'])*100
insulin['other_pct'] = (insulin['otherhi']/insulin['total_price'])*100


# make hhincome in scale 10000s
insulin['hhincome'] = insulin['hhincome']*10000

                   overall  no_insurance_dummy  primmedicare  primmedicaid  \
female         4688.000000          198.000000   2254.000000    811.000000   
nonhisp_white  4166.000000          120.000000   1852.000000    395.000000   
nonhisp_black  2206.000000           86.000000   1111.000000    356.000000   
hisp           2038.000000          177.000000    869.000000    418.000000   
other_race      594.000000            8.000000    243.000000     74.000000   
age              61.000000           53.000000     69.000000     51.000000   
hhincome          3.272050            2.800000      2.520000      1.927500   
diabetes_age     46.000000           39.000000     50.000000     38.000000   
self_ex          69.895000          165.170000     69.850000      3.000000   
monthcost        10.000000           25.193125      8.829143      0.327273   
avgmonthquant    15.000000           13.043478     14.393939     15.000000   
primpay         183.807377           25.193125    191.311224    

## 4. Machine Learning Analysis

### 4.1 LASSO

First, I filtered the data to only include those whose primary insurance is Medicare. The total number of observation is 4,075. I, then, split the observations into two groups such that 80% of the observations are randomly assigned to the training data set and the remaining 20% to the testing data set.

Then, I scaled data two different ways using MinMax scaler and Standard scaler. I do so because the scales of the data are vastly different which impact the final predictability and interpretation of the prediction. I scaled the data two different way to assess their performance and pick the scaler that offers the best out-of-sample performance.

Finally, I used k-fold validation to autptune the hyperparameter, alpha, in the LASSO. I set the k value to 3 with 10 splits. The minimum value of alpha is set to 0.1 with the upper value of 200 and increment of each iteration increases the alpha value by 0.1 (high increment due to computational limitation). The optimal alpha value is 22.7.

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from itertools import combinations
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# filter for only Medicare
insulin = insulin[insulin['medicare_y_dummy']==1]

# LASSO
## Define target variables and features
feature_list = ['lself_ex', 'self_ex', 'age', 'female' , 'nonhisp_white',
                'nonhisp_black', 'hisp', 'other_race', 'medicare_y_dummy', 
                'medicaid_y_dummy', 'private_y_dummy', 'other_dummy', 'no_insurance_dummy',
                'medicaid_x', 'medicare_x','private_x', 'otherhi', 'strength', 'hhincome',
                'heart_disease', 'hypertension', 'obesity', 'lipidem', 'self_pct',
                'medicare_pct','medicaid_pct','private_pct','other_pct', 'primmedicare',
                'primmedicaid','primprivate', 'primother', 'sumdays', 'sumquant']
target = 'avgmonthquant'

y = insulin[target]

Xmat = insulin[feature_list]

X = np.array(Xmat)

## Train test split
X_train,X_test,y_train,y_test= train_test_split(Xmat,y,test_size=0.2,random_state=3)

# scale data
# use minMax scaler
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train_mm = min_max_scaler.fit_transform(X_train)
X_test_mm = min_max_scaler.transform(X_test)
Xmat_mm = min_max_scaler.transform(Xmat)

# use standardscaler
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
X_train_ss = standard_scaler.fit_transform(X_train)
X_test_ss = standard_scaler.fit_transform(X_test)
Xmat_ss = standard_scaler.transform(Xmat)

## K-Fold validation to tune alpha in LASSO
model = Lasso()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0.1, 200, 0.1) # try increase upper bound
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(Xmat, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

# perform the search MM
results_mm = search.fit(Xmat_mm, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config_mm: %s' % results.best_params_)

# perform the search ss
results_mm = search.fit(Xmat_mm, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config_ss: %s' % results.best_params_)

MAE: -4.978
Config: {'alpha': 15.3}
MAE: -5.953
Config_mm: {'alpha': 0.1}
MAE: -5.953
Config_ss: {'alpha': 0.1}


#### LASSO: Unstandardized

I applied the optimal alpha of 15.3 and perform LASSO regression on unscaled data. LASSO regression suggests including 7 variables: out-of-pocket expenditure, amount paid by Medicaid, amount paid by Medicare, amount paid by other insurance, household income, number of day supply across survey year, and insulin units filled across survey year.

The R-square value is 0.02 which indicates that the model fits the data very poorly. This could be due to a number of factors, including the scale of the variables.

Next, I performed LASSO regression on scaled data to compare the R-square values.

In [9]:
lasso = Lasso(alpha=15.3)
# As alpha goes to zero, then the regession is closer to OLS.
# The higher the alpha, the less complex the model.
lasso.fit(X_train,y_train)
train_score=lasso.score(X_train,y_train)
test_score=lasso.score(X_test,y_test)
coeff_used = np.sum(lasso.coef_!=0)
print(coeff_used)
# Count the number of non-zero coefficients

print(test_score)
print(lasso.coef_)

8
0.023969927695677407
[ 0.00000000e+00  6.61438815e-04 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
  0.00000000e+00  0.00000000e+00  2.13012642e-04  1.23333085e-04
  6.63159062e-04 -0.00000000e+00  6.41203424e-06 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -1.33058654e-03
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -2.42500460e-02  4.16131868e-02]


#### LASSO: MinMax Scaler

I used MinMax scaler to scale the data because the scales for each variables were very different which could affect the fit of the model. I used the same algorithm to tune the alpha hyperparameter. The suggested alpha is 0.1 which may suggest that the lower bound during tuning was too low and that, after using MinMax scaler, LASSO regression is as good as ordinary least square in terms of the fit.

LASSO regression included 7 variables: age, female, non-hispanic black, hispanic, strength, heart disease, lipid metabolic diseases, percentage of total price paid by medicare, number of day supply across survey year, and insulin units filled across survey year. The R-sqaure is 0.01 which is worse than the unscaled data. In addition, the included variables may hinder the interpretability of the model, given the objective to predict changes in 30-day consumption in response to the change in out out-of-pocket (OOP) expenditure. This model does not include the OOP variable.

Next, I scaled the data using standard scaler for comparison.

In [10]:
lasso_mm = Lasso(alpha=0.1)
# As alpha goes to zero, then the regession is closer to OLS.
# The higher the alpha, the less complex the model.
lasso_mm.fit(X_train_mm,y_train)
train_mm_score=lasso_mm.score(X_train_mm,y_train)
test_mm_score=lasso_mm.score(X_test_mm,y_test)
coeff_used = np.sum(lasso_mm.coef_!=0)
print(coeff_used)
# Count the number of non-zero coefficients

print(test_mm_score)
print(lasso_mm.coef_)

7
0.011075465708769938
[ 0.00000000e+00  0.00000000e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00 -9.28632473e-02  0.00000000e+00  0.00000000e+00
  0.00000000e+00 -0.00000000e+00  9.96558279e-03 -8.35902169e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.99018437e+00
  0.00000000e+00 -0.00000000e+00  0.00000000e+00 -0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  8.96098546e-01 -0.00000000e+00
 -3.71077472e+01  7.68755901e+01]


#### LASSO: Standard Scaler

I used standard scaler to scale the data and follow the same protocal as outlined earlier. The suggested alpha is still 0.1 which may suggest that the lower bound during tuning was too low and that LASSO regression is as good as ordinary least square in terms of the fit.

LASSO regression included 13 variables: log of OOP, OOP, non hispanic black, hispanic, covered under Medicare, covered under private insurance, covered under other insurance, amount paid by Medicaid, amount paid by Medicare, strength, household income, heart disease, lipid metabolic diseases, number of day supply across survey year, and insulin units filled across survey year. The R-sqaure is 0.033 which is a slight improvement from the MinMax scaled data. Nonetheless, the fit of the model is still extremely poor.

In [11]:
lasso_ss = Lasso(alpha=0.1)
# As alpha goes to zero, then the regession is closer to OLS.
# The higher the alpha, the less complex the model.
lasso_ss.fit(X_train_ss,y_train)
train_ss_score=lasso_ss.score(X_train_ss,y_train)
test_ss_score=lasso_ss.score(X_test_ss,y_test)
coeff_used = np.sum(lasso_ss.coef_!=0)
print(coeff_used)
# Count the number of non-zero coefficients

print(test_ss_score)
print(lasso_ss.coef_)

13
0.02397486307860275
[ 5.98994169e-02  4.05486204e-01 -0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -0.00000000e+00  1.92918282e-02  1.44445609e-01
  0.00000000e+00  0.00000000e+00  0.00000000e+00 -4.91457447e-01
  0.00000000e+00  0.00000000e+00  6.31921265e-01 -0.00000000e+00
  7.49971062e-01 -0.00000000e+00  1.37512270e-01 -0.00000000e+00
 -0.00000000e+00  0.00000000e+00  0.00000000e+00 -6.59318882e-01
  0.00000000e+00 -0.00000000e+00  0.00000000e+00  0.00000000e+00
 -0.00000000e+00 -1.94070371e-03  4.41067898e-01 -0.00000000e+00
 -7.84527306e+00  9.90837104e+00]


### 4.2 Ridge

Next, I performed Ridge regression for comparison.

#### Ridge: Unscaled data

The R-square is 0.02 which is not an improvement from the LASSO regression on unscaled data. The fit is still extremely poor.

In [12]:
# Normal
ridge = Ridge(alpha=0.5)
ridge.fit(X_train,y_train)
train_score=ridge.score(X_train,y_train)
test_score=ridge.score(X_test,y_test)
coeff_used = np.sum(ridge.coef_!=0)
print(test_score)
print(ridge.coef_)

0.024916826232902478
[ 1.05049107e-01  9.78261586e-04 -2.56884498e-03  7.97752673e-02
 -4.02625186e-01 -2.88550567e-01  1.72336834e-02  6.73942069e-01
  0.00000000e+00  7.95681349e-02 -4.31989931e-02 -2.42454185e+00
  0.00000000e+00  9.19884071e-05  1.54322448e-04 -5.78860816e-05
  6.82225959e-04 -2.84043892e-01  5.45729476e-06 -1.12134582e-01
 -9.12268347e-02 -2.67296362e-01  1.61870481e-01 -2.67489249e-02
  4.64004250e-03  1.32763200e-02 -1.78456913e-03  1.06171323e-02
 -1.96992243e-01 -1.76031726e+00  2.34096125e+00 -3.83651746e-01
 -2.49143786e-02  4.25579210e-02]


#### Ridge: Min Max scaler

The R-square is 0.02, no improvement from the unscaled data.

In [13]:
# MM
ridge_mm = Ridge(alpha=0.5)
ridge_mm.fit(X_train_mm,y_train)
train_mm_score=ridge_mm.score(X_train_mm,y_train)
test_mm_score=ridge_mm.score(X_test_mm,y_test)
coeff_used = np.sum(ridge_mm.coef_!=0)
print(test_mm_score)
print(ridge_mm.coef_)

0.02343658757086442
[ 1.06094964e+00  6.90338965e+00 -2.53918327e-01  3.49795605e-02
 -3.90374579e-01 -3.22660524e-01  2.18420641e-02  6.91193039e-01
  0.00000000e+00  5.24670035e-02 -4.02882395e-02 -2.46860452e+00
  0.00000000e+00  2.26222994e+00  1.04029049e+01 -1.45966011e+00
  2.34292465e+01 -2.63456759e-01  2.42367543e+00 -8.66056775e-02
 -1.10832027e-01  1.49472418e-01  1.85122800e-01 -2.67151028e+00
  4.51548589e-01  9.83647808e-01 -1.90026150e-01  1.42634004e+00
 -2.38549554e-01 -1.49294470e+00  2.31628900e+00 -5.84794746e-01
 -1.00908348e+02  1.36085583e+02]


#### Ridge: Standard scaler

The R-square is 0.02. There is also no improvement fro mthe unscaled or Min Max scaled data. The fit is on par with that from LASSO.

In [14]:
# SS
ridge_ss = Ridge(alpha=0.5)
ridge_ss.fit(X_train_ss,y_train)
train_ss_score=ridge_ss.score(X_train_ss,y_train)
test_ss_score=ridge_ss.score(X_test_ss,y_test)
coeff_used = np.sum(ridge_ss.coef_!=0)
print(test_ss_score)
print(ridge_ss.coef_)

0.02466720745949702
[ 0.22923713  0.49926427 -0.02616341  0.03978111 -0.11451884 -0.050668
  0.07509422  0.20576796  0.          0.03720723 -0.01986927 -0.61238706
  0.          0.05786526  0.71349855 -0.10472765  0.84564512 -0.07915249
  0.2341827  -0.05523303 -0.03522805 -0.02716073  0.07336548 -0.68165947
  0.20111136  0.18377655 -0.03757873  0.29991098 -0.16430552 -0.30606484
  0.6039077  -0.18068764 -8.28260216 10.23261609]


### 4.3 Elastic Net

For comparison, I performed Elastic Net which is another shinkage method. First, I tune the hyperparameter, alpha, and the L1 ratio which determines the balance between L1 and L2 regularization methods.

The optimal alpha for the unscaled data is 0.801. For MinMax scaled data, the optimal alphas are 0.01 and 0.201, respectively. The optimal L1 ratio is 0.8 for all data.

In [15]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

# ELASTIC NET
model = ElasticNet()
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# define grid
grid = dict()
grid['alpha'] = np.arange(0.001, 1, 0.2)
grid['l1_ratio'] = np.arange(0, 1, 0.2)
# define search
search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
# perform the search
results = search.fit(Xmat, y)
# summarize
print('MAE: %.3f' % results.best_score_)
print('Config: %s' % results.best_params_)

# perform the search
results_mm = search.fit(Xmat_mm, y)
# summarize
print('MAE_mm: %.3f' % results_mm.best_score_)
print('Config_mm: %s' % results_mm.best_params_)

# perform the search
results_ss = search.fit(Xmat_ss, y)
# summarize
print('MAE_ss: %.3f' % results_ss.best_score_)
print('Config_ss: %s' % results_ss.best_params_)

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.856e+06, tolerance: 4.028e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.861e+06, tolerance: 4.038e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.866e+06, tolerance: 4.050e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.864e+06, tolerance: 4.044e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.594e+05, tolerance: 4.066e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.591e+05, tolerance: 4.055e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.596e+05, tolerance: 4.057e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.525e+05, tolerance: 4.057e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.454e+05, tolerance: 4.059e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.554e+05, tolerance: 4.048e+02
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider

...

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.946e+06, tolerance: 4.066e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.069e+06, tolerance: 2.287e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.947e+06, tolerance: 4.037e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.957e+06, tolerance: 4.050e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.959e+06, tolerance: 4.051e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.083e+06, tolerance: 2.288e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.963e+06, tolerance: 4.043e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent(
/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.971e+06, tolerance: 4.061e+02 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of 

MAE_ss: -5.315
Config_ss: {'alpha': 0.201, 'l1_ratio': 0.8}


#### EN: Unscaled data

Applying EN to unscaled data yielded R-square of 0.02 which is, again, not an improvement from LASSO or Ridge. The train MAE is 4.1 and the test (out-of-sample) MAE is 7.3. Given that the mean 30-day consumption is approximately 14 units for Medicare memebers, EN produces poor prediction for unscaled data with the error threshold being 35% of the mean values.

In [16]:
elas = ElasticNet(alpha=0.801, l1_ratio=0.8)
elas.fit(X_train, y_train)
train_score=elas.score(X_train,y_train)
test_score=elas.score(X_test,y_test)
print(test_score)

# Evaluate the model on training and test sets using Mean Absolute Error
train_predictions = elas.predict(X_train)
train_mae = mean_absolute_error(y_train, train_predictions)
print("Train Mean Absolute Error:", train_mae)

test_predictions = elas.predict(X_test)
test_mae = mean_absolute_error(y_test, test_predictions)
print("Test Mean Absolute Error:", test_mae)

0.02484633311968021
Train Mean Absolute Error: 4.144305382573323
Test Mean Absolute Error: 7.265982484330489


#### EN: Min Max scaler

EN has a much lower R-square value when applied to data that is scaled using Min Max. The R-square value is 0.01. The MAE for training data is 4.1 and for testing data is 8.9. Once again, EN has a poor prediction performance for Min Max scaled data.

In [17]:
## MM

elas_mm = ElasticNet(alpha=0.01, l1_ratio=0.8)
elas_mm.fit(X_train_mm, y_train)
train_score_mm=elas_mm.score(X_train_mm,y_train)
test_score_mm=elas_mm.score(X_test_mm,y_test)
print(test_score_mm)

# Evaluate the model on training and test sets using Mean Absolute Error
train_predictions_mm = elas_mm.predict(X_train_mm)
train_mae_mm = mean_absolute_error(y_train, train_predictions)
print("Train Mean Absolute Error MM:", train_mae_mm)

test_predictions_mm = elas_mm.predict(X_test_mm)
test_mae_mm = mean_absolute_error(y_test, test_predictions_mm)
print("Test Mean Absolute Error MM:", test_mae_mm)

0.010667192003643855
Train Mean Absolute Error MM: 4.144305382573323
Test Mean Absolute Error MM: 8.915973891622675


#### EN: Standard scaler

When applying EN to standard scaled data, the fit does not improve, evident by the R-square value of 0.02. The MAE values do not improve for neither the training nor the testing data set, compared to the unscaled and Min Max scaled data.

In [18]:
## SS

elas_ss = ElasticNet(alpha=0.201, l1_ratio=0.8)
elas_ss.fit(X_train_ss, y_train)
train_score_ss=elas_ss.score(X_train_ss,y_train)
test_score_ss=elas_ss.score(X_test_ss,y_test)
print(test_score_ss)

# Evaluate the model on training and test sets using Mean Absolute Error
train_predictions_ss = elas_ss.predict(X_train_ss)
train_mae_ss = mean_absolute_error(y_train, train_predictions)
print("Train Mean Absolute Error ss:", train_mae_ss)

test_predictions_ss = elas_ss.predict(X_test_ss)
test_mae_ss = mean_absolute_error(y_test, test_predictions_ss)
print("Test Mean Absolute Error ss:", test_mae_ss)

0.021426634993445992
Train Mean Absolute Error ss: 4.144305382573323
Test Mean Absolute Error ss: 7.385070952969643


### 4.4 Principal Component Analysis

Next, I performed PCA to further evaluate the prediction performance for algorithms that assumes linearity. I applied PCA to standardized data with 12 principal components. The R-square is -0.001 which is the poorest compared to LASSO, Ridge, and EN. This suggests that PCA is not an appropriate algorithm to make prediction for average 30-day insulin consumpotion amongst Medicare members. In addition, the nature of PCA does not allow for direct interpretation of variables. Therefore, it cannot be used for predicting the insulin consumption.

In [19]:
# PCA
# first normalize data
sc = StandardScaler()
x = sc.fit_transform(Xmat)

# now run PCA on the normalized data
pca = PCA(n_components = 12)
X_reduced = pca.fit_transform(x)
explained_variance = pca.explained_variance_ratio_
print(explained_variance)

# Now we try to develop a regression model on the components
X_train,X_test,y_train,y_test=train_test_split(X_reduced,y,test_size=0.2,random_state=3)
reg = LinearRegression()
reg.fit(X_train, y_train)
test_score=reg.score(X_test,y_test)
print(test_score)

[0.13920891 0.10194059 0.08169677 0.0728661  0.06466966 0.04810806
 0.04495343 0.0393004  0.03565792 0.03251857 0.03124164 0.03079118]
-0.001174097810465069


### 4.5 Random Forest

Evidently, algorithms that assume linearity performs poorly in terms of prediction. This raises the suspicion of non-linear relationship between the average 30-day insulin consumption and the predictors. As such, I try to utlize non-parametric approaches suchas random forest and neural network.

Setting up for Random Forest because the feature list changes from the analyses above.

In [31]:
# set up for running algorithms
feature_list = ['self_ex', 'medicare_x', 'medicaid_x', 'private_x','otherhi',
                'age','hispanic', 'hhincome', 'diabetes_age',  'heart_disease', 'obesity',
       'hypertension', 'lipidem', 'female',
       'nonhisp_white', 'nonhisp_black', 'hisp', 'other_race', 'sumquant', 'sumdays',
        'month', 'monthcost', 'strength', 'total_price',
       'self_pct']
target = 'avgmonthquant' 
y = insulin[target].to_numpy()
X = insulin[feature_list].to_numpy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# scale data
# use minMax scaler
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train_mm = min_max_scaler.fit_transform(X_train)
X_test_mm = min_max_scaler.transform(X_test)

# use standardscaler
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
X_train_ss = standard_scaler.fit_transform(X_train)
X_test_ss = standard_scaler.fit_transform(X_test)

#### RF: Unscaled data

In [32]:
# random forest no scale
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators = 20, max_depth=30, min_samples_leaf = 30, random_state = 0)
model_rf = rf.fit(X_train, y_train)
prediction=model_rf.predict(X_test)
from sklearn.metrics import r2_score
print('R2 Value:',r2_score(y_test, prediction))

R2 Value: 0.5285669790894393


#### RF: Min Max Scaler

In [33]:
# random forest no mm
rf_mm = RandomForestRegressor(n_estimators = 10, max_depth=15, min_samples_leaf = 15, random_state = 0)
model_rf_mm = rf_mm.fit(X_train_mm, y_train)
prediction_mm=model_rf_mm.predict(X_test_mm)
from sklearn.metrics import r2_score
print('R2 Value mm:',r2_score(y_test, prediction_mm))

R2 Value mm: 0.5649486066915983


#### RF: Standard scaler

In [34]:
# random forest no ss
rf_ss = RandomForestRegressor(n_estimators = 20, max_depth=30, min_samples_leaf = 30, random_state = 0)
model_rf_ss = rf_ss.fit(X_train_ss, y_train)
prediction_ss=model_rf_ss.predict(X_test_ss)
from sklearn.metrics import r2_score
print('R2 Value ss:',r2_score(y_test, prediction_ss))

R2 Value ss: 0.7774022394724587


#### RF: Prediction

Using RF with standard scaler produces the best out-of-sample prediction with the R-square value of 0.77, compared to 0.53 and 0.56 for unscaled and Min Max scaled data, respectively. I chose to use RF with standard scaler to predict the change in insulin coonsumption per 30-day for Medicare beneficiaries.

To do so, I constructed a pseudo data set which include individuals who are currently paying over \\$35 for 30-day supply. The reason for excluding those paying less than $35 is that those people will most likely not participate in this program. The pseudo data set is the replica of the real test data set we used in this analysis but I replaced all of the OOP expenditure with \\$35 which is the maximum amount OOP under SSM.

In [35]:
# make a pseudo data set for 35 oop
y_test35 = pd.DataFrame(y_test)
X_test35 = pd.DataFrame(X_test)
xy_test35 = pd.concat([y_test35, X_test35], axis = 1)
xy_test35.columns = ['avgmonthquant','self_ex','medicare_x', 'medicaid_x', 'private_x','otherhi',
                'age','hispanic', 'hhincome', 'diabetes_age',  'heart_disease', 'obesity',
       'hypertension', 'lipidem', 'female',
       'nonhisp_white', 'nonhisp_black', 'hisp', 'other_race', 'sumquant', 'sumdays',
        'month', 'monthcost', 'strength', 'total_price',
       'self_pct']
# Filter out paying less than, keep people paying 35
xy_test35 = xy_test35[xy_test35['monthcost']>35]
# scale psuedo data
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
xy_test35 = standard_scaler.fit_transform(xy_test35)
# make a df of pseudo data
xy_test35 = pd.DataFrame(xy_test35)
xy_test35.columns = ['avgmonthquant','self_ex','medicare_x', 'medicaid_x', 'private_x','otherhi',
                'age','hispanic', 'hhincome', 'diabetes_age',  'heart_disease', 'obesity',
       'hypertension', 'lipidem', 'female',
       'nonhisp_white', 'nonhisp_black', 'hisp', 'other_race', 'sumquant', 'sumdays',
        'month', 'monthcost', 'strength', 'total_price',
       'self_pct']
# split out y_test35
y_test35 = xy_test35['avgmonthquant']
# convert back to array
y_test35 = np.array(y_test35)
# split out the rest
x_columns = ['self_ex','medicare_x', 'medicaid_x', 'private_x','otherhi',
                'age','hispanic', 'hhincome', 'diabetes_age',  'heart_disease', 'obesity',
       'hypertension', 'lipidem', 'female',
       'nonhisp_white', 'nonhisp_black', 'hisp', 'other_race', 'sumquant', 'sumdays',
        'month', 'monthcost', 'strength', 'total_price',
       'self_pct']
X_test35 = xy_test35[x_columns]
# change month cost to 35
X_test35['monthcost'] = 35
#convert back to array
X_test35 = np.array(X_test35)

/var/folders/sm/w4wbgz91529dqchlbwz3d97r0000gn/T/ipykernel_32207/1397095312.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test35['monthcost'] = 35


After constructing the peudo data set, I fitted the RF model to predict the average 30-day insulin consumption. Then, I took the mean and median of the consumption before and after the SSM to compare the differences. There is a slight increase in the mean and median after SSM which supports the objective of the SSM which aimed to increase consumption and medication adherence.

However, the independent t-test suggests that the difference in the mean is not statistically signifcant with the t-statistics of -0.8 and the p-value of 0.4. This may be due to the small size sample and the fact that the differences are very small. However, we should keep in mind that medication consumption may not follow the classical demand-price relationship because consumption of medication, at a certain point, depends on the medical need of an individual. Therefore, changes in price may not necessarily result in drastic change in insulin consumption. Nonetheless, the predicted increase provide a valuable insight into the effectiveness of price cap on necessary medication such as insulin.

In [36]:
# we predict for comparison
# Convert X_test_ss to df
xy_test35 = pd.DataFrame(xy_test35)
xy_test35.columns = ['avgmonthquant','self_ex','medicare_x', 'medicaid_x', 'private_x','otherhi',
                'age','hispanic', 'hhincome', 'diabetes_age',  'heart_disease', 'obesity',
       'hypertension', 'lipidem', 'female',
       'nonhisp_white', 'nonhisp_black', 'hisp', 'other_race', 'sumquant', 'sumdays',
        'month', 'monthcost', 'strength', 'total_price',
       'self_pct']
xy_test35 = xy_test35.drop(columns=['avgmonthquant'])
# Add a column of prediction
xy_test35['predicted'] = model_rf_ss.predict(xy_test35)
# Mean and median for predicted test
print('Test data avgmonthquant mean:', xy_test35['predicted'].mean())
print('Test data avgmonthquant median:', xy_test35['predicted'].median())
# fit data to the 35 pseudo data set
X_test35_df = pd.DataFrame(X_test35)
X_test35_df['predicted'] = model_rf_ss.predict(X_test35)
# Mean and median for predicted 35
print('35 oop data avgmonthquant mean:', X_test35_df['predicted'].mean())
print('35 oop data avgmonthquant median:', X_test35_df['predicted'].median())

# test if the means are statistically different
# Perform independent t-test
from scipy import stats
t_statistic, p_value = stats.ttest_ind(xy_test35['predicted'], X_test35_df['predicted'])

# Print the results
print("t-statistic:", t_statistic)
print("p-value:", p_value)

Test data avgmonthquant mean: 17.160314969907844
Test data avgmonthquant median: 15.519812583937448
35 oop data avgmonthquant mean: 17.778787907239515
35 oop data avgmonthquant median: 16.091934845497004
t-statistic: -0.769002053666091
p-value: 0.4423142634431845


/Users/parimagphanthong/anaconda3/lib/python3.11/site-packages/sklearn/base.py:457: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


## 5. Conclusion

The objective of this project is to use a variety of machine learning algorithms to predict the change in average 30-day insulin consumption among Medicare beneficiaries, if SSM is implemented. I utilized parametric approaches including LASSO regression, Ridge regression, Elastic Net, and PCA. I also utilized non-parametric approach, Random Forest. I performed the analyses on data set that is scaled differently. This is because the scales of each variables are drastically different. I suspected that the difference in the scales may cause the prediction performance to be extremely poor so scaling using different scalers can address the issue.

I found that parametric approaches performed very poorly in predicting insulin demand out of sample. The table below compares the performance of LASSO and Ridge regression in terms of their R-sqaure values. Across all scaling techniques, LASSO and Ridge regression performed extremely poor with R-sqaure values as low as 0.011, meaning that the model can only explain 1% of the varibility in the average 30-day consumption. Furthermore, PCA performed poorly as well, despite my attempt to tune the hyperparameter and L1 ratio. The MAE from Elastic Net was as high as 8.92 in Min Max scaled data, and 7.27 and 7.39 for unscaled and standard scaled data, respectively. Given that the mean average 30-day consumption is approximately 14 among this set of samples, MAE value as high as 8.92 suggests that Elastic Net is also a poor-performing predictor. Another approach I attempted was PCA which had the R-square value of -0.001, suggesting an extrememly poor predictability. The extrememly poor performances across scaling techniques for these parametric approaches suggests that the relationship between consumption and predictors may not be linear.

A non-parametric approach confirms that the relationship between the average 30-day consumption and other included predictors is not linear. Random Forest can predict the change in consumption with accuracy as high as 77% in the standardized data, and 52.9% and 56.5% in unscaled and Min Max scaled data, respectively. Although this is not an impressive accuracy rate, it is much better than the parametric approaches. Using Random Forest, I found that SSM can increases the average consumption by 0.61 unit per 30-day. However, the t-test suggests that the difference is not statistically significant which may stem from small sample size. Nonetheless, this finding provide a valuable insight into the impact of price on necessity consumption such as insulin.

Future study should further explore other non-parametric algorithm to improve the accuracy of the prediction. Algorithm such as Neural Network can potentially improve predictability significantly due to its complex and delicate network. I could not perform Neural Network due to technical issue. In addition, future study should tune hyperparameters in the non-parametric model to improve predictability.

| Algorithm     | Metric        | Unscaled Data  | Min Max Scaler  | Standard Scaler | 
| ------------- | ------------- | -------------- | --------------- | --------------- |   
| LASSO         | $R^2$         | .024           | 0.011           | .024            |
| Ridge         | $R^2$         | .025           | 0.023           | .025            |
| Elastic Net   | MAE           | 7.27           | 8.92            | 7.39            |
| PCA           | $R^2$         | -              | -               | -.001               |
| Random Forest | $R^2$         | .529           | 0.565           | .777            |
